In [1]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

## Assignment 4

## Task 1 

Implement probability of selection for single objective optimization. * (refer eqn.
12, 13, 14)

In [3]:
np.set_printoptions(precision=4,suppress=True) # easier to check values rn when we dont care too much about the exactness

In [196]:
# in minimization think probability of being wrong, in max probability of acceptance
# probability of sample A dominating sample B in max is P(A > B) is
def prob_selection(a, b, sigma_a, sigma_b):
    #sigma_b = sigma_b + 0.01 # just to prevent division by zero
    # seems not to be the correct way of hanlding sigma 0
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    top = -2.5*m / np.sqrt(2 + 2*s**2)
    bot = 1 + np.exp(top)
    return 1 / bot

# b clearly dominates
a = np.array([1.0, 1.0])
b = np.array([0.5, 1.5])
sigma_a = .01
sigma_b = .01

prob_selection(a, b, sigma_a, sigma_b)

array([1., 0.])

In [195]:
def prob_sel2(a, b, sigma_a, sigma_b):
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    bot = 0.8*np.sqrt(2 + 2*s**2)
    return 0.5*(1 + np.tanh(m/bot))
    
    
    
# a1 clearly dominates b1, b1 clearly dominates a2
a = np.array([1.0, 1.0])
b = np.array([0.5, 1.5])
sigma_a = 0.001
sigma_b = 0.001

prob_sel2(a, b, sigma_a, sigma_b)

array([1., 0.])

So we can see that probability of being wrong with first items (1 and 2) by choosing a is 0.22. Again for second items (1,5) thats 0.007.  And also with 3rd items (3,1) its 0.92. So we can clearly see its working properly.

## Task 2

Implement probability of dominance and ranking. * (refer eqn. 15, 17, 18, 22)

- Generate 50 samples from DTLZ2 problem (3 objectives, 5 decision variables) using LHS sampling
- Build Kriging surrogates using the generated samples
- Resample or generate 20 data points randomly
- Rank the 20 points using probability of dominance ranking

In [145]:
dtlz2_3 = test_problem_builder("DTLZ2", n_of_variables=5, n_of_objectives=3)

In [146]:
def create_samples(dec_dim, samples):
    # create samples 
    x = lhs(dec_dim, samples)     
    return np.asarray(x)

In [147]:
x = create_samples(5, 50)
print(x.shape)
eval_results = dtlz2_3.evaluate(x)
y = eval_results.objectives
print(y[:5])

(50, 5)
[[0.0015 0.001  1.2205]
 [1.102  0.1826 0.3468]
 [0.107  0.0588 1.4862]
 [0.0478 1.0128 0.4683]
 [1.0729 0.381  0.7594]]


In [148]:
print(y[:,0])

[0.0015 1.102  0.107  0.0478 1.0729 0.3648 0.2215 0.194  0.6316 0.4401
 0.5976 1.0801 0.6267 0.1289 0.0709 0.4962 0.1839 0.6767 0.1372 1.1737
 0.1433 0.4147 0.5981 0.6159 0.5137 0.5745 0.1024 0.7553 0.3949 0.7295
 0.0104 0.2927 0.024  0.7091 0.461  0.0893 0.8149 0.6307 1.2136 1.1422
 1.1002 0.6958 0.4623 0.3283 0.9385 0.1827 0.919  0.2595 1.117  0.0041]


In [149]:
## build surrogates

kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 20.0), nu=1.5)
    
y1 = y[:,0]
gpr1 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y1)

y2 = y[:,1]
gpr2 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y2)

y3 = y[:,2]
gpr3 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y3)

In [150]:
## resampling

x_samples = create_samples(5, 20)

# predict
y1_sample, y1_std = gpr1.predict(x_samples, return_std=True) 

# predict
y2_sample, y2_std = gpr2.predict(x_samples, return_std=True) 

# predict
y3_sample, y3_std = gpr3.predict(x_samples, return_std=True) 

In [1070]:


# eq 15: MO P(A < B) : product to k, P(Aj < Bj)
def smaller(a, b):
    p = 0
    # probl sel calculates P(A>B), so turn around for P (A < B)
    props = prob_selection(b, a, .01, .01)
    p = np.prod(props)
    #p = props
    #print(props)

    return p

# eq 17: MO P(A > B) : product to k, 1 - P(Aj < Bj)
def bigger(a, b):
    return 1 - smaller(a, b)


# eq 18: MO P(A = B) : 1 - P(A < B) - P(A > B)
def equals(a, b):
    return 1 - smaller(a, b) - bigger(a, b)


def me_equals(a, b):
    summa = 0
    
    #if f1 < f2:
    
def dominates(x: np.ndarray, y: np.ndarray):
    dom = 0

    for i in range(len(x)):
        if x[i] > y[i]:
            return 0
        elif x[i] < y[i]:
            dom = 1
            
    return dom

In [1071]:
dominates(a,b)

1

In [1072]:
a = np.array([0.35,0.5])
b = np.array([0.85,0.5])


B = a
E = b

smaller(B,E)

0.5

In [1073]:
bigger(B,E)

0.5

In [1074]:
bigger(a,b)

0.5

In [1075]:
equals(a,b)

0.0

In [1076]:
f = bigger(a, b)

# objective values from fitness measurements
f1 = np.array([0,1])
f2 = np.array([1,2])
# f1 dominates f2

In [1077]:
rrr = np.array([1.,2.,3.])
kkk = np.array([4.,5.,6.])



In [1078]:
def sum_of_rank_pos(n):
    return n*(n - 1)/2

sum_of_rank_pos(ff.shape[0])

15.0

In [1080]:

# eq 21: give fitnesses, return rankings 
def eq21(f):
    ri = np.zeros_like(f)
    for i in range(f.shape[0]):
        for j in range(f.shape[0]):
            ri[i] += smaller(f[j], f[i])
            #ri[i] += dominates(f[j], f[i])
        ri[i] -= 0.5
    return ri
    

# this is wrong or the equals atleast is wrong
def eq22(sols):
    print(sols.shape)
    num_sols = len(sols)
    ri = np.zeros(6)
    # could vectorize do stupid first
    print(ri.shape)
    #n = f.shape[0]
    for i in range(num_sols):
        #print(sols[i])
        for j in range(num_sols):
            ri[i] += dominates(sols[i], sols[j])    
            print(f"ri {ri[i]} __ {i} ___ {j}")
    
        ri[i] -= 0.5
    ri *= 0.5
            #ri[i] -= 0.5
        
            
    print(ri)
    #ri -= 0.5
    return ri
    
    

# eq 22: proba ranking : sum to n, P(Fj > Fi) + 0.5 sum to n P(Fj = Fi) - 0.5
def probabilitistic_ranking():
    ranks = 0
    #non_dom = 
    
    return ranks

In [1081]:
A = [0.2, 0.9]  # tässä on a1 a2. kun lasketaan prod P (Ai < bi) i to n, tulee P(A < B)
B = [0.3, 0.5]
C = [0.5, 0.3]
D = [0.8, 0.2]
E = [0.85, 0.5]
F = [0.9, 0.95]

sols = np.array([A, B, C, D, E, F])
print(sols.shape)
ff = np.zeros(sols.shape[0])
# this being eq21
for i in range(ff.shape[0]):
    for j in range(ff.shape[0]):
        ff[i] += smaller(sols[j], sols[i])
    #ff[i] -= 0.5

(6, 2)


In [1082]:
ff
#pr1 = ff

array([0.25  , 0.25  , 0.25  , 0.25  , 2.7481, 5.2461])

In [1083]:
pr1 = eq21(ff)
pr1

array([1.5002, 1.4998, 1.4998, 1.5002, 4.    , 5.    ])

In [1084]:
pr2 = eq22(sols)
pr2

(6, 2)
(6,)
ri 0.0 __ 0 ___ 0
ri 0.0 __ 0 ___ 1
ri 0.0 __ 0 ___ 2
ri 0.0 __ 0 ___ 3
ri 0.0 __ 0 ___ 4
ri 1.0 __ 0 ___ 5
ri 0.0 __ 1 ___ 0
ri 0.0 __ 1 ___ 1
ri 0.0 __ 1 ___ 2
ri 0.0 __ 1 ___ 3
ri 1.0 __ 1 ___ 4
ri 2.0 __ 1 ___ 5
ri 0.0 __ 2 ___ 0
ri 0.0 __ 2 ___ 1
ri 0.0 __ 2 ___ 2
ri 0.0 __ 2 ___ 3
ri 1.0 __ 2 ___ 4
ri 2.0 __ 2 ___ 5
ri 0.0 __ 3 ___ 0
ri 0.0 __ 3 ___ 1
ri 0.0 __ 3 ___ 2
ri 0.0 __ 3 ___ 3
ri 1.0 __ 3 ___ 4
ri 2.0 __ 3 ___ 5
ri 0.0 __ 4 ___ 0
ri 0.0 __ 4 ___ 1
ri 0.0 __ 4 ___ 2
ri 0.0 __ 4 ___ 3
ri 0.0 __ 4 ___ 4
ri 1.0 __ 4 ___ 5
ri 0.0 __ 5 ___ 0
ri 0.0 __ 5 ___ 1
ri 0.0 __ 5 ___ 2
ri 0.0 __ 5 ___ 3
ri 0.0 __ 5 ___ 4
ri 0.0 __ 5 ___ 5
[ 0.25  0.75  0.75  0.75  0.25 -0.25]


array([ 0.25,  0.75,  0.75,  0.75,  0.25, -0.25])

In [1085]:
res = pr1 + pr2
res

array([1.7502, 2.2498, 2.2498, 2.2502, 4.25  , 4.75  ])

In [1086]:
np.sum(res)

17.5

In [432]:
assert np.floor(np.sum(res)) == sum_of_rank_pos(ff.shape[0])